In [3]:
# !pip install tensorflow
# !pip  install moabb
# !pip install imblearn

In [31]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import os, moabb,sys, time,pickle
from moabb.datasets import BNCI2014001
import pandas as pd
from cnn_utils import *
import warnings
warnings.filterwarnings('ignore')
import tensorflow.compat.v1 as tf
import numpy as np
import scipy.io as siobn 
from moabb.pipelines.utils import FilterBank
tf.disable_v2_behavior()
from sklearn.pipeline import make_pipeline,Pipeline
from pyriemann.estimation import Covariances

##########

# saving the results
def save_csvfile(results,sfile_name):
    if(os.path.isfile('./'+sfile_name) ==True):
        print("exists")
        results.to_csv(sfile_name, mode="a", index=False, header=False)
    else:
        print("No.. so creating")
        results.to_csv(sfile_name,  index=False, header=True) 
    return

from pyriemann.tangentspace import TangentSpace,FGDA
def fgda(X_train,X_test,y_train):
    fb =  FilterBank(make_pipeline(Covariances(estimator="oas"),FGDA()), flatten=False)
    fb.fit(X_train,y_train)
    xt=fb.transform(X_train);xe=fb.transform(X_test)
    return xt,xe,y_train

def load_pkl(sub,nband,fpath):
    direc=fpath #+"/xy/sub"+str(sub)
    with open(direc+'xt_cnn7.pkl', 'rb') as file:
        xt=pickle.load(file)
    with open(direc+'xe_cnn7.pkl', 'rb') as file:
        xe=pickle.load(file)
    with open(direc+'yt_cnn7.pkl', 'rb') as file:
        yt=pickle.load(file)
    with open(direc+'ye_cnn7.pkl', 'rb') as file:
        ye=pickle.load(file)
    print(xt.keys())
    return xt[(sub,nband)],xe[(sub,nband)],yt[(sub,nband)],ye[(sub,nband)]

In [32]:
def model(x, w, w2, w3, w_o, p_keep_conv, p_keep_hidden,b1,b2):
    l1a = tf.nn.relu(tf.nn.conv3d(x, w, strides=[1, 1, 1, 1, 1], padding='VALID'))
    l1a = tf.nn.bias_add(l1a, b1)
    l1a = tf.nn.dropout(l1a, p_keep_conv)

    l2a = tf.nn.relu(tf.nn.conv3d(l1a, w2, strides=[1, 1, 1, 1, 1], padding='VALID'))
    l2a = tf.nn.bias_add(l2a, b2)
    l2a = tf.nn.dropout(l2a, p_keep_conv)
      

    l2 = tf.reshape(l2a, [-1, w3.get_shape().as_list()[0]])
    l3 = tf.nn.relu(tf.matmul(l2, w3))
    l3 = tf.nn.dropout(l3, p_keep_hidden)

    pyx = tf.matmul(l3, w_o)
    return pyx

def nn_cnn3d(trX,trY,teX,teY,itr,k1,k2,f1,f2,eta,dropout):
    nfb=trX.shape[-1]
    nch=trX.shape[1]
    n_out=trY.shape[-1]
    trX = trX.reshape(-1, nch, nch, nfb, 1) 
    teX = teX.reshape(-1, nch, nch, nfb, 1) 

    x = tf.placeholder("float", [None, nch, nch, nfb, 1]) 
    y_ = tf.placeholder("float", [None, n_out])

    kernel1=k1
    kernel2=k2
    feature_map1=f1 #50
    feature_map2=f2 #100

    w = tf.get_variable("w", shape=[kernel1, kernel1, kernel2, 1, feature_map1],
                        initializer=tf.truncated_normal_initializer(stddev=0.01))
    w2 = tf.get_variable("w2", shape=[nch+1-kernel1, nch+1-kernel1, nfb+1-kernel2, feature_map1, feature_map2],
                         initializer=tf.truncated_normal_initializer(stddev=0.01))
    w3 = tf.get_variable("w3", shape=[feature_map2, feature_map2], initializer=tf.truncated_normal_initializer(stddev=0.01))
    w_o = tf.get_variable("w_o", shape=[feature_map2, n_out], initializer=tf.truncated_normal_initializer(stddev=0.01))

    b1 = tf.get_variable("b1", shape=[feature_map1], initializer=tf.constant_initializer(0.0))
    b2 = tf.get_variable("b2", shape=[feature_map2], initializer=tf.constant_initializer(0.0))

    p_keep_conv = tf.placeholder("float")
    p_keep_hidden = tf.placeholder("float")
    y = model(x, w, w2, w3, w_o, p_keep_conv, p_keep_hidden,b1,b2)

    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1)), tf.float32))
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
    train_op = tf.train.AdamOptimizer(eta).minimize(cost)
    predict_op = tf.argmax(y, 1)

    acc = [0] * itr
    tracc = [0] * itr
    # Launch the graph in a session
    ypred={}
    with tf.Session() as sess:
        tf.initialize_all_variables().run()
        for i in range(itr):
            sess.run(train_op, feed_dict={x: np.array(trX), y_: np.array(trY), p_keep_conv: dropout, p_keep_hidden: dropout})
            #print(100 * np.mean(np.argmax(teY, axis=1) == sess.run(predict_op, feed_dict={x: teX, y_: teY, p_keep_conv: 1.0, p_keep_hidden: 1.0})))
    #         st = time.time()

            ypred[i]=sess.run(predict_op, feed_dict={x: teX, y_: teY, p_keep_conv: 1.0, p_keep_hidden: 1.0})
            acc[i] = np.mean(np.argmax(teY, axis=1) == ypred[i])
            #en = time.time()
            tracc[i] = sess.run(accuracy, feed_dict={x: trX, y_: trY, p_keep_conv: 0.8, p_keep_hidden: 0.8})
            print(i,tracc[i],acc[i])
    y_pred=ypred[np.argmax(acc)]        
    return y_pred,ypred[i]#,y_pred #,ypred[np.argmax(acc)]



In [ ]:
nbs=["4"] #["2468","4","24","246"]
subj = [ 1,2,3,4,5,6,7,8,9]

nbands_Sel=[7] #[2,3,4,5,6,8,9,10]
mode="baseline_fb_fixed7_fgda"
dataset = BNCI2014001();dataset_name="BNCI2014001"
itr=200;k1=5;k2=5;f1=50;f2=100;eta=0.001;dropout=0.6 #itr=200;k1=5;k2=5;f1=50;f2=100;eta=0.001;dropout=0.7

pipe="cnn3d_itr"+str(itr)+"_k1"+str(k1)+"_k2"+str(k2)+"_f1"+str(f1)+"_f2"+str(f2)+"_eta"+str(eta)+"_dropout"+str(dropout)
for nb in nbs:
    fpath="tw_fb"+nb+"/"#fpath="timewin_filters_all/fb"+nb
    fpath="baseline_fb/"
    yp={};y_act_p={}
    for sub in subj:
        for nband in nbands_Sel:
            print("bands_hz:",nb,"|| subject:",sub, "|| nband:",nband)
            xt,xe,yt,ye=load_pkl(sub,nband,fpath)
            print(xt.shape,xe.shape,yt.shape,np.unique(yt))
            xt,xe,yt=fgda(xt,xe,yt)
            trX = xt;      teX = xe
            trY,teY=onehot_encode_list(yt,ye)
            tf.reset_default_graph()
            tf.set_random_seed(1230)
            yp[sub],y_act_p[sub]=nn_cnn3d(trX,trY,teX,teY,itr,k1,k2,f1,f2,eta,dropout)
         

In [ ]:
df = pd.DataFrame(columns=['dataset','session','channels','subject','n_filters','pipeline','acc',
    'recall','precision','f1','kappa','sensitivity','specificity'])
sfile_name = fpath+"CNN_accResults_CrossSess_"+mode+"_dscore_nbands"+nb+".csv"    
nchan=xt.shape[1]
for sub in subj:
    acc,recall,precision,f1,kappa,sens,spec=all_metrics(ye,yp[sub])
    df.loc[len(df)] = [pipe, "session_E", nchan, sub, nband, "3DCNN", acc,recall,precision,f1,kappa,sens,spec]
save_csvfile(df,sfile_name)

In [ ]:
df = pd.DataFrame(columns=['dataset','session','channels','subject','n_filters','pipeline','acc',
    'recall','precision','f1','kappa','sensitivity','specificity'])
sfile_name = fpath+"CNN_act_accResults_CrossSess_"+mode+"_dscore_nbands"+nb+".csv"    
nchan=xt.shape[1]
for sub in subj:
    acc,recall,precision,f1,kappa,sens,spec=all_metrics(ye,y_act_p[sub])
    df.loc[len(df)] = [pipe, "session_E", nchan, sub, nband, "3DCNN", acc,recall,precision,f1,kappa,sens,spec]
save_csvfile(df,sfile_name)